In [27]:
import os
import cython
%load_ext cython
import pandas as pd
import numpy as np
import functions as f
import re
import sys
from skimage.transform import resize
from skimage.draw import disk
import time
from scipy.optimize import curve_fit
import re
from PIL import Image
import orientationpy

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [26]:
#first check that the folders are correct
print(sys.path[3])

M:\tnw\bn\gk\GL\Papers\2022 - Membrane with Felix and Djim\Codes\AFM_orientation_analysis


In [25]:
def load_AFM_txt(n):
    '''
    Loads a AFM image from a txt file. 
    Parameters
    ----------
    n : int
        DESCRIPTION. Index number of the image to open in the config file. 

    Returns
    -------
    imAFM: numpy.ndarray
        DESCRIPTION. Region of the original image determined by the pixels in dim. 
    dim: dict
        DESCRIPTION. dimensions of the image. 
    px_size: float
        DESCRIPTION. Pixel size of the image in micrometers per pixel. 
    config['name'][indexConfig]: str
        DESCRIPTION. file name.
    width: float
        DESCRIPTION. Width of the original image      
    '''
    global indexConfig
    indexConfig = n
    #Set the folder where the txt file is
    folderAFM = os.path.join(os.path.normpath(sys.path[3] + "/" + "/input"))
    # load config file
    global config
    config = f.loadingMod.LoadConfigFile()
    # get the crop dimensions saved in the config file and stored them in a dict
    ymin,ymax,xmin,xmax=[int(s) for s in re.findall(r'\b\d+\b',config['crop'][indexConfig])]
    dim={"ymin":ymin, "ymax":ymax, "xmin":xmin, "xmax": xmax}
    # get the width of the image stored in the txt file of the image
    with open(folderAFM+"//"+config['name'][indexConfig]) as fp:
        for i, line in enumerate(fp):
            if i == 1:
                width = float(line.split(" ")[2])
                break
    #get the original image size in px stored in the file name
    pattern = re.compile(r'(\d+)px')
    match = pattern.search(config['name'][indexConfig])
    px_value = int(match.group(1))
    #calculate the pixel size
    px_size = width/px_value
    #load AFM image
    imAFM = f.loadingMod.LoadTxtFileFromConfig(config, fileFolder = folderAFM, iConfig=indexConfig)
    return imAFM, dim, px_size, config['name'][indexConfig], width

def pre_orientation_AFM(n):
    '''
    Convert float image into 8 bit.
    For the orientation information we need a 8 bit downscaled figure where low intenisty correlates with the background. This step might not be necessary however it removes computational time and allows the use of any 8 bit image tool. 

    Parameters
    ----------
    n : int
        DESCRIPTION. Index number of the image to open in the config file. 

    Returns
    -------
    imAFMp: numpy.ndarray
        DESCRIPTION. Crop of the original image in 8 bit. 
    px_size: float
        DESCRIPTION. Pixel size of the image in micrometers per pixel. 
    name: str
        DESCRIPTION. file name.
    width: float
        DESCRIPTION. Width of the original image      
    '''
    #load crop of the AFM image
    imAFM, dim, px_size, name, width = load_AFM_txt(n)
    #gets the dimensions of the image
    S = [dim['xmax']-dim['xmin'],dim['ymax']-dim['ymin']]
    #check whether image has NaN values
    has_nans = np.isnan(imAFM).any()
    if has_nans:
        #if the image has NaN values, skip it. 
        print("Image data contains NaN values")
    else:
        #if not, transform it to 8 bit. 
        imAFMp = f.preprocessingMod.FloatImgTo8Bit(im=imAFM,
                                                   size=(S[1], S[0]),
                                                   config=config,
                                                   iConfig=n
                                                   )
    return imAFMp, px_size, name, width

def orientation_AFM(n):
        '''
        Calculates orientation image.
            
        Parameters
        ----------
        n : int
            DESCRIPTION. Index number of the image to open in the config file. 
    
        Returns
        -------
        orientationsAFM: dict
            DESCRIPTION.  orientations dict with numpy.ndarray for the ['theta'],['coherency'],['energy']  
        px_size: float
            DESCRIPTION. Pixel size of the image in micrometers per pixel. 
        name: str
            DESCRIPTION. file name.
        width: float
            DESCRIPTION. Width of the original image      
        imAFMp: numpy.ndarray
            DESCRIPTION. Crop of the original image in 8 bit.  
        '''
        #open 8bit AFm image
        imAFMp, px_size, name, width = pre_orientation_AFM(n)
        # compute structure tensor
        imAFMT = f.analysingMod.CalculateStructureTensor(imAFMp,
                                                         mode="gaussian",
                                                         )
        #compute orientations
        orientationsAFM = f.analysingMod.CalculateOrientations(imAFMT,
                                                               mask=False
                                                               )
        #save in a text file each of the images produced by the analysis: orientation, energy, and coherency. 
        save_folder = os.path.join(os.path.normpath(sys.path[3] + "/output"))
        for i in orientationsAFM.keys():
            name_file = str(n) + "_" + i + ".txt"
            np.savetxt(save_folder + "\\" + name_file, orientationsAFM[i], delimiter="\t")
        
        print("==== Complete the orientation for the AFM txt file! ====")

        return orientationsAFM, px_size, name, width, imAFMp

def pre_orientation_SIM(folder_dir, name):
    '''
    loads simulations images and convert them into 8 bit.
    
    Parameters
    ----------
    folder_dir: path
        DESCRIPTION. path of folder where image is stored. 
    name: str
        DESCRIPTION. name of png file

    Returns
    -------
    imSIMp: numpy.ndarray
        DESCRIPTION. image in 8 bit. 
    '''
    #load simulation image
    imSIM = f.loadingMod.LoadPngFile(name, folder_dir)
    # transform the background to be "dark"
    imSIM =  255- imSIM
    #check whether image has nans
    has_nans = np.isnan(imSIM).any()
    if has_nans:
        #if it does, skip image
        print("Image data contains NaN values")
    else: #If it does not
        #resize image to speed up the analysis
        imSIM_resized = resize(imSIM, (768,768))
        #transform it to 8 bit. 
        imSIMp = f.preprocessingMod.FloatImgTo8Bit(im=imSIM_resized,
                                                   size=(imSIM_resized.shape[1], imSIM_resized.shape[0]),
                                                   # size = (768, 768)
                                                   )
        #save resized 8 bit image
        im_save = Image.fromarray(imSIMp).convert('RGB')
        absolute_path_I = os.path.join(folder_dir,name[:-4]+"_resized_invert.png")
        im_save.save(absolute_path_I)
    return imSIMp

def orientation_SIM(name, type, run):
        '''
        Calculates orientation image.
            
        Parameters
        ---------- 
        name: str
            DESCRIPTION. name of png file
        type: str
            DESCRIPTION: type of simulation --> crossing ot alignment. 
        run: str
            DESCRIPTION: run number as "run"+"number"
        Returns
        -------
        orientationsSIM: dict
            DESCRIPTION.  orientations dict with numpy.ndarray for the ['theta'],['coherency'],['energy']  
        px_size: float
            DESCRIPTION. Pixel size of the image in micrometers per pixel. 
        name: str
            DESCRIPTION. file name.
        width: float
            DESCRIPTION. Width of the original image      
        imSIMp: numpy.ndarray
            DESCRIPTION. Original image in 8 bit.  
        '''
        #Put together the input parameters to determine the file name. 
        folder_dir = os.path.join(os.path.normpath(sys.path[3] + "/" + "/input"+"/"+type+"/"+run))
        #open image
        imSIMp = pre_orientation_SIM(folder_dir, name)
        #set width or the image, which is always 5. 
        width = 5
        #calculate pixel size 
        px_size = width/imSIMp.shape[1]
        #compute structure tensor
        imSIMt = f.analysingMod.CalculateStructureTensor(imSIMp,
                                                         mode="gaussian",
                                                         )
        #compute orientations
        orientationsSIM = f.analysingMod.CalculateOrientations(imSIMt,
                                                               mask=False
                                                               )
        
        #save in a text file each of the images produced by the analysis: orientation, energy, and coherency. 
        save_folder = os.path.join(os.path.normpath(sys.path[3] + "/output"))

        for i in orientationsSIM.keys():
            np.savetxt(save_folder + "\\" + type + "_" + run + "_" + name[:-4]+"_"+str(i)+".txt", orientationsSIM[i], delimiter="\t")

        print("==== Complete the orientation for the AFM txt file! ====")

        return orientationsSIM, px_size, name, width, imSIMp

def dot_unit(x, y):
    '''
    Returns the dot product of two angles or an angle and an array of angles in radians (considering they are unit vectors).
    Parameters
    ---------- 
    x: float
        DESCRIPTION. value of the first angle 
    y: float or numpy.ndarray
        DESCRIPTION: if float, it is the second angle; and if numpy.ndarray, it is the array of angles  
    Returns
    -------
    (np.cos(x)*np.cos(y))+(np.sin(x)*np.sin(y)): float or numpy.ndarray
            DESCRIPTION.  Result of the dot product 
    '''
    return ((np.cos(x)*np.cos(y))+(np.sin(x)*np.sin(y)))

def ring_mask(array, center, diameter, ring_size=3):
    '''
    Creates a ring-shaped mask of the image "array", centered in the pixe "center", with an inner diameter of "diameter", and a thickness of "ring_size", 
    Parameters
    ---------- 
    array: numpy.ndarray
        DESCRIPTION. image to which the ring mask will be applied  
    center: numpy.ndarray, tuple, or list
        DESCRIPTION. central pixel of the ring
    diamater: int  
        DESCRIPTION. Inner diameter of the ring
    ring_size: int
        DESCRIPTION. Thickness of the ring. 
        The default is 3
    Returns
    -------
    array: numpy.ndarray
        DESCRIPTION. original image (array) masked with the created ring. 
    '''
    #Calculate the total diameter of the ring
    new_size = diameter+ring_size
    #create circular mask with a radius of the inner diameter of the ring over 2
    rr, cc = disk((center[0], center[1]), (diameter / 2), shape=array.shape)
    #create a circular mask with a radius of the total diameter over 2
    rr2, cc2 = disk((center[0], center[1]), (new_size / 2), shape=array.shape)
    #create a new empty image (all 0) with the dimensions of the original image
    mask = np.zeros(shape=array.shape, dtype=np.uint8)
    #set to a value of 1 the area inside the second (bigger) circular mask. 
    mask[rr2,cc2] = 1
    #set back to a value of 0 the area inside the first (smaller) circular mask 
    mask[rr,cc] = 0
    #transform the mask into a bool
    mask = mask > 0
    #apply mask on the image
    array[~mask] = np.nan
    return array

def select_random_pixels(image, diameter, number=5000):
    '''
    Selects "number" ammount of random pixels from "image". Select pixels that are further than "diameter" pixels away from the edge of the image. 
    Parameters
    ---------- 
    image: numpy.ndarray
        DESCRIPTION. image.   
    diamater: int  
        DESCRIPTION. Distance from the edge within which no pixels will be selected. 
    number: int
        DESCRIPTION. number of random pixels to select 
        The default is 5000
    Returns
    -------
    array: numpy.ndarray
        DESCRIPTION. array containing the indices (x and y location) of each of the pixels.  
    '''
    # Create a grid of indices
    rows, cols = np.indices(image.shape)
    
    # Compute the distance from each pixel to the nearest edge
    distances_to_edges = np.stack([rows, cols, image.shape[0] - rows - 1, image.shape[1] - cols - 1])
    
    # Compute the minimum distance along the third axis
    distances_to_nearest_edge = np.min(distances_to_edges, axis=0)
    
    # Create a binary mask based on the distance from the circular region around the edges
    mask = distances_to_nearest_edge < diameter
    
    # Apply the mask to the original image
    result = np.where(mask, 0, image)
    
    #calculate how many random pixels are selected. 
    valid_pixels = np.argwhere(result != 0)
    #create empty array where the pixels will be stored. 
    random_pixels = np.empty([0, 0])
    if valid_pixels.shape[0] <= 0:
        #if there are no pixels selected, give errors
        print('No pixels available!')
    elif valid_pixels.shape[0] < number:
        #if there are pixels, but less than the ammount asked, save them, but wive a warning error. 
        print("less than", number, 'pixels avaliable, using', valid_pixels.shape[0], "pixels")
        number = valid_pixels.shape[0]
        random_indices = np.random.choice(valid_pixels.shape[0], size=number, replace=False)
        random_pixels = valid_pixels[random_indices]
    else:
        #if enough pixels have been selected, save them
        random_indices = np.random.choice(valid_pixels.shape[0], size=number, replace=False)
        random_pixels = valid_pixels[random_indices]
    return random_pixels

def FitFunction(x,C,L,C2):
    '''
    Fit function for the orientational decay
    Parameters
    ---------- 
    x: float
        DESCRIPTION. Distance from the central pixel.   
    C: float 
        DESCRIPTION. Initial value. 
    L: float
        DESCRIPTION. Characteristic patch size, or lambda 
    C2: float
        DESCRIPTION. Saturation value at which the decay stops. 
    Returns
    -------
    (C2+((C-C2)*(np.exp(-x/L)))): float
        DESCRIPTION. Output value  
    '''
    return (C2+((C-C2)*(np.exp(-x/L))))

def fit_AFM (x, y):
    '''
    Fitting of the orientational decay to the fit function
    Parameters
    ---------- 
    x: numpy.ndarray
        DESCRIPTION. Array containing the distances corresponding to each value in y.   
    y: numpy.ndarray 
        DESCRIPTION. Values of the orientational decay.  
    Returns
    -------   
    C: float 
        DESCRIPTION. Initial value. 
    L: float
        DESCRIPTION. Characteristic patch size, or lambda 
    C2: float
        DESCRIPTION. Saturation value at which the decay stops. 
    rmse_fit: float
        DESCRIPTION. Root-mean-square deviation between the observed orientational decay and the fitted data

    '''    
    #guess initial values used for the fit
    p0=[0.1, 5, 0.4]
    #fit the data to the function
    C,L, C2=curve_fit(FitFunction,x, y,p0)[0] 
    #calculate Root-mean-square deviation
    rmse_fit = np.sqrt(np.mean((y-FitFunction(x,C,L, C2))**2))
    return C, L, C2, rmse_fit

In [8]:
%%cython 
########working in cython#######
import numpy as np
cimport numpy as np
import time
import functions.gerard_functions as fg
# @cython.boundscheck(False)
cpdef dict calculate_dot_plot_cy(np.ndarray data, np.ndarray selection, int ring_size = 3):
    '''
    Calculate the orientational decay data 
    Parameters
    ---------- 
    data: numpy.ndarray
        DESCRIPTION. Orientation image.   
    selection: numpy.ndarray 
        DESCRIPTION. Selection of random pixels.
    ring_size: thickness of the ring  
    Returns
    -------   
    results: dict 
        DESCRIPTION. dictionary containing central pixel as key and with values containing the orientational decay for that pixel.  

    '''    
    #initialize variable in cython
    cdef int x, y, i
    cdef dict results
    cdef double cent_px, start_time, end_time, start_time_t, end_time_t
    cdef list results_px
    cdef np.ndarray p_dot, p_dot_masked, angles
    #create the dict to save the data
    results = {}
    #start counting time
    start_time_t = time.time()
    #for each central pixel in selection
    for xy in selection:
        #save each of coordinate of the pixel. 
        x, y =  xy[0], xy[1]
        start_time = time.time()
        #create a list to save the results
        results_px = []
        print(x,y)
        #transform the orientation image to radians
        angles = np.radians(np.copy(data))
        #get the value of the central pixel
        cent_px = np.copy(angles[x, y])
        #compute the dot product of the central pixel with the whole image. 
        p_dot = fg.dot_unit(cent_px, angles)
        #for each inner diameter of the ring (starting in 2, until 150, in steps of the ring thickness)
        for i in range(2, 150, ring_size):
            #mask the image with the specific ring
            p_dot_masked = fg.ring_mask(np.copy(p_dot), (x,y), i, ring_size)
            #append the average of each ring into the list with resuklts
            results_px.append(np.nanmean((p_dot_masked)))
        #and save that list to the dictonary when finished. 
        results[(x,y)]= results_px
        end_time = time.time()
        print("time:", end_time-start_time, "sec")
    end_time_t = time.time()
    print("time:", end_time_t-start_time_t, "sec")
    return results

In [14]:
#set of indices of the config file containing manually filtered AFM images of hexamers with bundle network, octamers with filament networks, and hexamers with filament networks (top to bottom)
AFM_hex_bundle = [5, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24,32, 33, 34, 35, 36, 37, 38, 40, 42, 43]
AFM_oct_fil = [44, 45, 46, 47, 48, 49, 50, 51]
AFM_hex_fil = [28, 29, 31,52, 53, 54, 55, 56, 57, 58]
#initialize pandas dataframe to save the data
results_df = pd.DataFrame(columns=['name', "pixel_size",'selected pixels',"averages",'lambda',"C1", "C2", "RMSE_fit", 'av_E','std_E','av_C','std_C'])
#for each AFM image to analyze
for i in AFM_hex_bundle:
    #get orientation image
    data, px_size, name, width, original = orientation_AFM(i)
    angles = np.copy(data['theta'])
    #select random pixels
    selection = select_random_pixels(angles, 73, 5000)
    #and if the selection of pixels has more than 0, and the image is big enough (4um in width) 
    if selection.size>0 and width>=4:
        #calculate the orientational decay for each pixel in the selection
        results = calculate_dot_plot_cy(angles, selection, 3)
        #average the orientational decay of all pixels
        averages = [sum(col) / float(len(col)) for col in zip(*list(results.values()))]
        #calulate the distances in micrometers
        dist = np.arange(px_size*2, len(averages) * px_size*3, px_size*3)
        #fit the orientational decay to the function
        C, L, C2, rmse_fit = fit_AFM(dist, averages)
        #save into the dataframe using the index of the config file
        results_df.loc[i] = [name, px_size, selection, averages,L, C, C2, rmse_fit, 
                             np.mean(np.copy(data['energy'])), np.std(np.copy(data['energy'])), 
                             np.mean(np.copy(data['coherency'])), np.std(np.copy(data['coherency']))]
        ##Save into csv file after each image analyzed. 
        #TODO: Change the file name for every list
        results_df.to_csv(path_or_buf=os.path.join(os.path.normpath(sys.path[3] + "/" + "/output/AFM_data_hex_bundle.csv")),  columns=['name', "pixel_size",'lambda',"C1", "C2", "RMSE_fit", 'av_E','std_E','av_C','std_C'], sep='\t')
    else:
        print("Image or region is too small")
        
        
# results_df
# angles.shape

KeyboardInterrupt: 

In [24]:
np.seterr(divide='ignore', invalid='ignore')
#make set of lists to loop over all the data
mode = ["alignment", "crossing"]
runs = ["run0", "run1", "run2", "run3", "run4", "run5", "run6", "run7", "run8", "run9", "run10"]
simulation_files = ["kp1E0kg1E0.png","kp1E0kg1E2.png","kp1E0kg1E4.png","kp1E2kg1E0.png","kp1E2kg1E2.png","kp1E2kg1E4.png",
                    "kp1E4kg1E0.png","kp1E4kg1E2.png","kp1E4kg1E4.png"]
#initialize dataframe 
results_df = pd.DataFrame(columns=['name',"log_kp", "log_kg", "pixel_size",'selected pixels',"averages",'lambda',"C1", "C2", "RMSE_fit", 'av_E','std_E','av_C','std_C', "mode", "run"])
#for each mode, run, and image
for m in range(len(mode)):
    for r in range(len(runs)):
        for i in range(len(simulation_files)):
            #make an ID to identify the image in the results
            id = int(str(m+1)+str(r)+str(i))
            #save the growth rate and the pairing rate
            kp = simulation_files[i][4]
            kg = simulation_files[i][-5]
            print(kp, kg)
            #get orientation image
            data, px_size, name, width, original = orientation_SIM(simulation_files[i], mode[m], runs[r])
            #select random pixels
            selection = select_random_pixels(original, 73, 5000)
            angles = np.copy(data['theta'])
            if selection.size>0 and width>=4:
                #calculate the orientational decay for each pixel in the selection
                results = calculate_dot_plot_cy(angles, selection, 3)
                #average the orientational decay of all pixels
                averages = [sum(col) / float(len(col)) for col in zip(*list(results.values()))]
                #calulate the distances in micrometers
                dist = np.arange(px_size*2, len(averages) * px_size*3, px_size*3)
                #fit the orientational decay to the function
                C, L, C2, rmse_fit = fit_AFM(dist, averages)
                #save into the dataframe using the index of the config file
                results_df.loc[id] = [name,kp, kg, px_size, selection, averages,L, C, C2, rmse_fit, 
                                     np.mean(np.copy(data['energy'])), np.std(np.copy(data['energy'])), 
                                     np.mean(np.copy(data['coherency'])), np.std(np.copy(data['coherency'])), mode[m], runs[r]]
                #Save into csv file after each image analyzed. 
                #TODO: Change the file name if needed
                results_df.to_csv(path_or_buf=os.path.join(os.path.normpath(sys.path[3] + "/"+ "/output/SIM.csv")),  columns=['name',"log_kp", "log_kg", "pixel_size",'lambda',"C1", "C2", "RMSE_fit", 'av_E','std_E','av_C','std_C', 'mode', 'run'], sep='\t')
            else:
                print("Image or region is too small")
            

0 0
==== Complete the orientation for the AFM txt file! ====
503 329
time: 0.32904720306396484 sec
373 594
time: 0.32416701316833496 sec
475 360
time: 0.3320784568786621 sec
134 193
time: 0.33315420150756836 sec
679 108
time: 0.3251163959503174 sec
370 457
time: 0.3370978832244873 sec
479 82
time: 0.34108757972717285 sec
642 402
time: 0.37000370025634766 sec
498 179
time: 0.3610401153564453 sec
578 499
time: 0.3540661334991455 sec
118 169
time: 0.34308695793151855 sec
362 197
time: 0.3819310665130615 sec
374 182
time: 0.36206817626953125 sec
145 152
time: 0.33606410026550293 sec
85 445


KeyboardInterrupt: 